In [ ]:
from azureml.core import Workspace, Experiment, Run
from sklearn.externals import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from azureml.contrib.explain.model.visualize import ExplanationDashboard
from azureml.explain.model.tabular_explainer import TabularExplainer
from azureml.contrib.explain.model.explanation.explanation_client import ExplanationClient
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", message='l1_reg="auto" is deprecated')

DATA_PATH = "../../data/"

In [ ]:
ws = Workspace.from_config()

experiment_name = 'diabetes_regression'
experiment = Experiment(workspace=ws, name=experiment_name)

In [ ]:
def find_best_run(experiment, metric, goal='minimize'):
    runs = {}
    run_metrics = {}

    # Create dictionaries containing the runs and the metrics for all runs containing the 'mse' metric
    for r in tqdm(experiment.get_runs(include_children=True)):
        metrics = r.get_metrics()
        if 'rmse' in metrics.keys():
            runs[r.id] = r
            run_metrics[r.id] = metrics
    
    if goal == 'minimize':
        min_run = min(run_metrics, key=lambda k: run_metrics[k][metric])
        return runs[min_run]
    else:
        max_run = max(run_metrics, key=lambda k: run_metrics[k][metric])
        return runs[max_run]

### Reload a previously trained model from our Run History

In [ ]:
hx_run = find_best_run(experiment, 'rmse')
client = ExplanationClient.from_run(hx_run)

In [ ]:
# Download previously captured model explanation
global_explanation = client.download_model_explanation()

### Download saved model file
```python
# You can also download a registered model
mdl = ws.models['diabetes_regression_model']
mdl.download('./model')```

In [ ]:
# Get the model file and download to a "model" folder
os.makedirs('model', exist_ok=True)

hx_run.download_file("outputs/model.pkl", "./model")

### Load the model into memory

In [ ]:
# Unpickle model
best_model = joblib.load('./model/model.pkl')

### Load a training/testing dataset

In [ ]:
diabetes_df = pd.read_csv(os.path.join(DATA_PATH,"diabetes.csv"))

diabetes_df.columns

y = diabetes_df.pop('target').values
X = diabetes_df.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print (f"Data contains {len(X_train)} training samples and {len(X_test)} test samples")

### Show Explanation Dashboard from the loaded explanation and model files

In [ ]:
ExplanationDashboard(global_explanation, best_model, X_test)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hx_run).show()